In [ ]:
import os
import pylab as plt
import numpy as np
import torch
import torch.nn
from data_loader import get_dataloader
from config import config as cfg
from models import VoxelNet
from utils import utils

from scipy.spatial.transform import Rotation as R

from lyft_dataset_sdk.lyftdataset import LyftDataset
from lyft_dataset_sdk.utils.data_classes import LidarPointCloud, Box, Quaternion
from lyft_dataset_sdk.utils.geometry_utils import view_points, transform_matrix
from lyft_dataset_sdk.eval.detection.mAP_evaluation import Box3D, recall_precision

%matplotlib inline

In [ ]:
!rm images
!rm maps
!rm lidar
!rm data
!ln -s /run/media/hoosiki/WareHouse1/mtb/datasets/lyft-3d-od/train/images images
!ln -s /run/media/hoosiki/WareHouse1/mtb/datasets/lyft-3d-od/train/maps maps
!ln -s /run/media/hoosiki/WareHouse1/mtb/datasets/lyft-3d-od/train/lidar lidar
!ln -s /run/media/hoosiki/WareHouse1/mtb/datasets/lyft-3d-od/train/data data

In [ ]:
lyft_dataset = LyftDataset(data_path='.', json_path='data', verbose=False)

In [ ]:
sample = lyft_dataset.get('sample', 'cea0bba4b425537cca52b17bf81569a20da1ca6d359f33227f0230d59d9d2881')
#sample = lyft_dataset.get('sample', '21f0543ffe5aa3886ab18b1b9eee6377fc525658ef92f449c3636245470fb6f0')
#sample = lyft_dataset.get('sample', '1d143ba709eaaf6a6db03a0e28d9a1665f76d6b2aa0b9816aef431a9cfbd8eca')
#sample = lyft_dataset.get('sample', 'c7f7de87ec90c8993d4e7d5463208d2aa9f5ecde671960536f39b9a86f939d3c')

lidar_info = lyft_dataset.get('sample_data', sample['data']['LIDAR_TOP'])
lidar_data_path = lyft_dataset.get_sample_data_path(sample['data']['LIDAR_TOP'])
gt_boxes3d = lyft_dataset.get_boxes(sample['data']['LIDAR_TOP'])

ego_pose = lyft_dataset.get('ego_pose', lidar_info['ego_pose_token'])
calibrated_sensor = lyft_dataset.get('calibrated_sensor', lidar_info['calibrated_sensor_token'])

In [ ]:
gt_boxes3d[0]

In [ ]:
gt_boxes3d[1]

In [ ]:
gt_boxes3d[-1]

In [ ]:
gt_boxes3d_xyzlwhr = np.array([[gt_box3d.center[0],
                                gt_box3d.center[1],
                                gt_box3d.center[2],
                                gt_box3d.wlh[1],
                                gt_box3d.wlh[0],
                                gt_box3d.wlh[2],
                                gt_box3d.orientation.yaw_pitch_roll[0]] for gt_box3d in gt_boxes3d])

In [ ]:
gt_boxes3d = utils.convert_boxes3d_from_global_to_sensor_frame(gt_boxes3d, ego_pose, calibrated_sensor)

In [ ]:
gt_boxes3d_xyzlwhr = np.array([[gt_box3d.center[0],
                                gt_box3d.center[1],
                                gt_box3d.center[2],
                                gt_box3d.wlh[1],
                                gt_box3d.wlh[0],
                                gt_box3d.wlh[2],
                                gt_box3d.orientation.yaw_pitch_roll[0]] for gt_box3d in gt_boxes3d])

In [ ]:
boxes3d = utils.convert_boxes3d_xyzlwhr_to_Box(gt_boxes3d_xyzlwhr)

In [ ]:
tmp = utils.convert_boxes3d_from_sensor_to_global_frame(boxes3d, ego_pose, calibrated_sensor)

In [ ]:
tmp[0]

In [ ]:
tmp[1]

In [ ]:
tmp[-1]

In [ ]:
tmp[-1].orientation.yaw_pitch_roll[0]